In [9]:
# Install the latest master of Haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-hd2r2310/farm-haystack_bcec6eb923ad41aebda38615c910621c
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-hd2r2310/farm-haystack_bcec6eb923ad41aebda38615c910621c
  Resolved https://github.com/deepset-ai/haystack.git to commit 1e01cd0efb20d98905c84461e0433d5db4084b7e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [10]:
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http, print_answers
from haystack.nodes import FARMReader, TransformersReader

In [11]:
# In-Memory Document Store
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore()

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0


In [12]:
doc_dir = "piaf-ota/data/processed"

docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

# print(docs[:3])
document_store.write_documents(docs)

INFO - haystack.utils.preprocessing -  Converting piaf-ota/data/processed/air_europa7.txt
INFO - haystack.utils.preprocessing -  Converting piaf-ota/data/processed/air_europa9.txt
INFO - haystack.utils.preprocessing -  Converting piaf-ota/data/processed/air_europa12.txt
INFO - haystack.utils.preprocessing -  Converting piaf-ota/data/processed/air_europa0.txt
INFO - haystack.utils.preprocessing -  Converting piaf-ota/data/processed/air_europa3.txt
INFO - haystack.utils.preprocessing -  Converting piaf-ota/data/processed/air_europa5.txt
INFO - haystack.utils.preprocessing -  Converting piaf-ota/data/processed/air_europa10.txt
INFO - haystack.utils.preprocessing -  Converting piaf-ota/data/processed/air_europa11.txt
INFO - haystack.utils.preprocessing -  Converting piaf-ota/data/processed/air_europa2.txt
INFO - haystack.utils.preprocessing -  Converting piaf-ota/data/processed/air_europa4.txt
INFO - haystack.utils.preprocessing -  Converting piaf-ota/data/processed/air_europa6.txt
INFO - 

In [13]:
from haystack.nodes import TfidfRetriever

retriever = TfidfRetriever(document_store=document_store)

INFO - haystack.nodes.retriever.sparse -  Found 13 candidate paragraphs from 13 docs in DB


In [14]:
reader = FARMReader(model_name_or_path="etalab-ia/camembert-base-squadFR-fquad-piaf", use_gpu=True)

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find etalab-ia/camembert-base-squadFR-fquad-piaf locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Automatically detected language from language model name: french
INFO - haystack.modeling.model.language_model -  Loaded etalab-ia/camembert-base-squadFR-fquad-piaf
INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.modeling.infer -  Got ya 71 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0

In [15]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [18]:
# You can configure how many candidates the reader and retriever shall return
# The higher top_k for retriever, the better (but also the slower) your answers.
prediction = pipe.run(
    query="peut-on amener son animal ?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]/opt/conda/lib/python3.9/site-packages/haystack/modeling/model/prediction_head.py:483: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.74 Batches/s]


In [17]:
# Now you can either print the object directly...
from pprint import pprint

pprint(prediction)

{'answers': [<Answer {'answer': 'Air Europa ne transportera que le passager dont le nom figure sur le billet', 'type': 'extractive', 'score': 0.9205362498760223, 'context': '\n**1.** **Dispositions générales**\n(a) Air Europa ne transportera que le passager dont le nom figure sur le billet, auquel il peut être demandé de pré', 'offsets_in_document': [{'start': 857, 'end': 932}], 'offsets_in_context': [{'start': 38, 'end': 113}], 'document_id': 'b5bde73c58d568b92b9f0bc42c7954c', 'meta': {'name': 'air_europa3.txt'}}>,
             <Answer {'answer': 'Chaque coupon de vol inclus dans le billet du passager est accepté pour le transport dans la classe de service', 'type': 'extractive', 'score': 0.5700965821743011, 'context': 'mentaire est dû.\n(d) Chaque coupon de vol inclus dans le billet du passager est accepté pour le transport dans la classe de service, à la date et au v', 'offsets_in_document': [{'start': 522, 'end': 632}], 'offsets_in_context': [{'start': 20, 'end': 130}], 'document_i